In [126]:
#!pip install seaborn 

import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
sns.__version__

'0.12.1'

In [127]:
FOLDER = '../scrapping/data'
FILE = '20-years-odds-20221209134405.csv'
LIMIT = 30000

data = pd.read_csv(f'{FOLDER}/{FILE}')
data.shape

(61119, 17)

In [128]:
data.head()

,home_team,away_team,correct_score,1N2_1,1N2_N,1N2_2,matchdate,period,location,season,competition,home_score,away_score,bet_won,target,odd_win,best_odd_won
0,Charlton,Liverpool,0:4,3.60,3.6,1.70,2001-05-19,2001-05,england,2001,premier-league-2000,0.0,4.0,2,2.0,1.70,True
1,Coventry,Bradford City,0:0,1.79,3.3,3.80,2001-05-19,2001-05,england,2001,premier-league-2000,0.0,0.0,N,1.0,3.30,False
2,Derby,Ipswich,1:1,3.50,3.3,1.96,2001-05-19,2001-05,england,2001,premier-league-2000,1.0,1.0,N,1.0,3.30,False
3,Everton,Sunderland,2:2,2.50,3.1,2.50,2001-05-19,2001-05,england,2001,premier-league-2000,2.0,2.0,N,1.0,3.10,False
4,Leeds,Leicester,3:1,1.27,4.5,9.99,2001-05-19,2001-05,england,2001,premier-league-2000,3.0,1.0,1,0.0,1.27,True


In [129]:
teams = [data.home_team.tolist()] + [data.away_team.tolist()]

teams = np.unique(teams[0])
len(np.unique(teams))

280

In [130]:
scores = np.unique(data.correct_score)
df_scores = pd.DataFrame(data.correct_score.value_counts(normalize=True))
#df_scores.columns = [['scores','ratio']]
#df_scores.sort_values(ascending=False)
df_scores.columns = ['ratio']
df_scores.sort_values('ratio',ascending=False)

,ratio
1:1,0.129158
1:0,0.108019
2:1,0.090823
0:0,0.085047
2:0,0.080090
...,...
4:7,0.000016
8:2,0.000016
9:2,0.000016
0:8,0.000016


# Baseline

Sinmply compute a baseline by taking the 1N2 odds, transpose them to 1/odds then scale date and train a multi-output regressor.

In [131]:
df = data[['1N2_1','1N2_N','1N2_2','home_score','away_score']]
df

,1N2_1,1N2_N,1N2_2,home_score,away_score
0,3.60,3.60,1.70,0.0,4.0
1,1.79,3.30,3.80,0.0,0.0
2,3.50,3.30,1.96,1.0,1.0
3,2.50,3.10,2.50,2.0,2.0
4,1.27,4.50,9.99,3.0,1.0
...,...,...,...,...,...
61114,6.85,4.03,1.53,2.0,2.0
61115,4.04,3.75,1.88,1.0,3.0
61116,1.29,5.53,10.81,4.0,0.0
61117,6.12,4.46,1.52,1.0,2.0


In [132]:
X = 1/df[['1N2_1','1N2_N','1N2_2']]
y = df.drop(['1N2_1','1N2_N','1N2_2'],axis=1)

In [133]:
X

,1N2_1,1N2_N,1N2_2
0,0.277778,0.277778,0.588235
1,0.558659,0.303030,0.263158
2,0.285714,0.303030,0.510204
3,0.400000,0.322581,0.400000
4,0.787402,0.222222,0.100100
...,...,...,...
61114,0.145985,0.248139,0.653595
61115,0.247525,0.266667,0.531915
61116,0.775194,0.180832,0.092507
61117,0.163399,0.224215,0.657895


In [134]:
# summarize dataset
print(X.shape, y.shape)

(61119, 3) (61119, 2)


In [135]:
X_train=X[:LIMIT]
X_test=X[LIMIT:]

y_train = y[:LIMIT]
y_test = y[LIMIT:]

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(30000, 3) (31119, 3) (30000, 2) (31119, 2)


In [136]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [137]:
model = LinearRegression(fit_intercept=False)

model = DecisionTreeRegressor()

model = KNeighborsRegressor(n_neighbors=7)

model.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=7)

In [138]:
row = X_test[:1]
row

array([[-1.01369666, -0.23505044,  1.07279154]])

In [139]:
model.predict(row)

array([[1.        , 1.14285714]])

In [140]:
y_test[:1]

,home_score,away_score
30000,0.0,1.0


In [141]:
yhat = model.predict(X_test)

In [142]:
yhat

array([[1.        , 1.14285714],
       [1.42857143, 1.42857143],
       [1.        , 2.        ],
       ...,
       [2.71428571, 0.42857143],
       [0.42857143, 0.85714286],
       [1.14285714, 1.57142857]])

In [143]:
eval = y_test.copy()

eval['home_pred']=yhat[:,0]
eval['away_pred']=yhat[:,1]

conditions = [
    ((eval['home_pred'] > eval['away_pred']) & (eval['home_score'] > eval['away_score'])),
    ((eval['home_pred'] == eval['away_pred']) & (eval['home_score'] == eval['away_score']))
]
choices = [True,True]

eval['prediction'] = np.select(conditions, choices, default=False)

In [144]:
eval.prediction.value_counts(normalize=True)

False    0.670555
True     0.329445
Name: prediction, dtype: float64